In [6]:
from __future__ import annotations
import os, json, re
import pandas as pd
from typing import List, Dict, Any
from datetime import datetime
from pathlib import Path


In [13]:
input_csv = "../data/bdp_data.csv"

In [16]:
# Mapeo de nombres lógicos -> columnas reales del CSV
DEFAULT_COLUMNS: Dict[str, str] = {
    # claves mínimas
    "fecha": "fecha",
    "hora": "hora",
    "notas": "notas",
    "estresores": "eventos_estresores",
    "tags": "tags",

    # áreas núcleo
    "animo": "animo",
    "activacion": "activacion",
    "sueno": "sueno_calidad",
    "conexion": "conexion",
    "proposito": "proposito",
    "claridad": "claridad",
    "estres": "estres",

    # métricas/variables extra
    "horas_sueno": "horas_sueno",
    "siesta_min": "siesta_min",
    "autocuidado": "autocuidado",
    "alimentacion": "alimentacion",
    "movimiento": "movimiento",
    "dolor_fisico": "dolor_fisico",
    "ansiedad": "ansiedad",
    "irritabilidad": "irritabilidad",
    "meditacion_min": "meditacion_min",
    "exposicion_sol_min": "exposicion_sol_min",
    "agua_litros": "agua_litros",
    "cafe_cucharaditas": "cafe_cucharaditas",
    "alcohol_ud": "alcohol_ud",
    "medicacion_tomada": "medicacion_tomada",
    "medicacion_tipo": "medicacion_tipo",
    "otras_sustancias": "otras_sustancias",
    "interacciones_significativas": "interacciones_significativas",
    "glicemia": "glicemia",
}

In [17]:
def _resolve_columns(df: pd.DataFrame, expected: Dict[str,str]) -> pd.DataFrame:
    """
    Intenta mapear/renombrar columnas reales a los nombres esperados en DEFAULT_COLUMNS,
    usando comparación insensible a mayúsculas/acentos/espacios.
    No elimina columnas; sólo renombra si encuentra equivalentes.
    """
    if df is None or df.empty:
        return df
    # Normaliza encabezados reales
    real_cols = list(df.columns)
    norm_to_real = {_norm(c): c for c in real_cols}
    rename_map = {}
    for k, exp in expected.items():
        if not isinstance(exp, str): 
            continue
        if exp in df.columns:
            continue  # ya está
        # buscar por forma normalizada
        real = norm_to_real.get(_norm(exp))
        if real:
            rename_map[real] = exp
        else:
            # sinónimo básico para fecha
            if k == "fecha":
                for alt in ["fecha", "fechas", "dia", "día", "date", "fecha_registro"]:
                    real = norm_to_real.get(_norm(alt))
                    if real and real not in rename_map:
                        rename_map[real] = exp
                        break
    if rename_map:
        df = df.rename(columns=rename_map)
    return df

In [18]:
df = pd.read_csv(input_csv, dtype={"hora": "string"})
df.columns = [str(c).strip() for c in df.columns]
# mapea a nombres esperados
df = _resolve_columns(df, DEFAULT_COLUMNS)
if "fecha" in df.columns:
    df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce", dayfirst=True)

if start_date:
    sd = pd.to_datetime(start_date, errors="coerce", dayfirst=True)
    df = df[df["fecha"] >= sd]
if end_date:
    ed = pd.to_datetime(end_date, errors="coerce", dayfirst=True)
    df = df[df["fecha"] <= ed]

if tag_filter and "tags" in df.columns:
    df = df[df["tags"].astype(str).str.contains(tag_filter, case=False, na=False)]

df.info()

NameError: name '_norm' is not defined